In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.0 kB/s 
     |████████████████████████████████| 438 kB 73.5 MB/s 
     |████████████████████████████████| 5.8 MB 57.0 MB/s 
     |████████████████████████████████| 1.6 MB 57.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

# IMPORTING NECESSARY LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from scipy import signal
import cv2
import os
import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# LOADING DATA

In [ ]:
labels = ['Negative', 'Positive'] 
img_size = 28
def read_images(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label) 
        class_num = labels.index(label)
        print(class_num)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE) 
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

Dataset = read_images('/content/drive/MyDrive/dataset/train')


0
1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


# FEATURE SCALING

In [ ]:
x = []
y = []

for feature, label in Dataset:
    x.append(feature) 
    y.append(label) 
        
x = np.array(x).reshape(-1,1, img_size, img_size)
x = x / 255

print(x.shape)
y = np.array(y)

(28718, 1, 28, 28)


# MODEL BUILDING

In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    
    def forward(self,input):
        pass
    
    def backward(self, output_gradient, learning_rate):
        pass

class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient
    
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))


class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)

    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i, j], "valid")
        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient
    
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)
        
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def forward(self, input):
        return np.reshape(input, self.output_shape)

    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)
    
def log_loss(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def log_loss_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)    
    

# IMPLEMENTING THE MODEL

In [ ]:
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

epochs =20
learning_rate =0.1

for e in range(epochs):
  error = 0
  for a,b in zip(x,y):
    output = a
    for layer in network:
      output = layer.forward(output)

    error+=log_loss(b,output)

    grad = log_loss_prime(b,output)
    for layer in reversed(network):
      grad = layer.backward(grad,learning_rate)

  error /= len(x)
  print(f"{e+1}/{epochs},error={error}")

for a,b in zip(x,y):
  output = a
  for layer in network:
    output = layer.forward(output)
  print(f"pred:{np.argmax(output)},true:{np.argmax(y)}")


Streaming output truncated to the last 5000 lines.
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:1

# LOADING TEST DATA AND FEATURE SCALING

In [ ]:
test_data = "/content/drive/MyDrive/dataset/test"

In [ ]:
x_test = []
y_test = []

for feature, label in Dataset:
    x_test.append(feature) 
    y_test.append(label) 
        
x_test = np.array(x_test).reshape(-1,1, img_size, img_size)
x_test = x_test / 255


y_test = np.array(y_test)

# IMPLEMENTING THE MODEL

In [ ]:
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

epochs =20
learning_rate =0.1

for e in range(epochs):
  error = 0
  for a,b in zip(x_test,y_test):
    output = a
    for layer in network:
      output = layer.forward(output)

    error+=log_loss(b,output)

    grad = log_loss_prime(b,output)
    for layer in reversed(network):
      grad = layer.backward(grad,learning_rate)

  error /= len(x_test)
  print(f"{e+1}/{epochs},error={error}")

for a,b in zip(x_test,y_test):
  output = a
  for layer in network:
    output = layer.forward(output)
  print(f"pred:{np.argmax(output)},true:{np.argmax(y_test)}")

Streaming output truncated to the last 5000 lines.
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:13818
pred:0,true:13818
pred:0,true:13818
pred:0,true:13818
pred:1,true:1